## 1 Импорт библиотек и модулей

In [1]:
!pip install -q ydata_profiling

In [5]:
%matplotlib inline

In [3]:
# для работы с таблицами
import pandas as pd

# для EDA анализа
import ydata_profiling

# для математических вычислений
import numpy as np

# для разделения данных на выборки
from sklearn.model_selection import train_test_split

# модель машинного обучения
from sklearn.linear_model import LinearRegression

# расчёта качества модели
from sklearn.metrics import r2_score

## 2 Подготовка данных
Считаем данные из csv-файла в датафрейм, сохраним в переменную `df` и проведём EDA анализ.

In [10]:
def read_eda(name):
    try:
        df = pd.read_csv('/datasets/' + name + '.csv')
    except:
        df = pd.read_csv('https://code.s3.yandex.net/datasets/' + name + '.csv')
    ydata_profiling.ProfileReport(df, progress_bar=False).to_file(name + '.html')
    return df

In [11]:
df = read_eda('insurance')

[Посмотреть анализ.](https://www.dropbox.com/s/sbru4dn9g3kez7b/insurance.html?dl=0)

**Проблем не обнаружено.**

Разобьём данные на признаки (`x`) и целевой признак (`y`).

In [7]:
x = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']

<div style="border:solid steelblue 5px; padding: 30px">
<p>
        <b>После изучения данных, стала ясна их общая структура:</b> 5000 строк, 5 столбцов, 2 типа данных: float64, int64.
    <br>
        <b>Аномалий, подлежащих обработке, обнаружено не было.</b>
    </p>  
</div>

## 3 Преобразование
### 3.1 Признаки умножают на обратимую матрицу, изменится ли качество линейной регрессии, при том, что её можно обучить заново?

***

**Обозначения:**

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

***

**Формулы:**

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

***

**Ответ:** b. Не изменится. 

***

**Обоснование:** 

Установим тождество между a и a'.

$$
a = Xw
$$

$$
a' = XPw'
$$

Укажем, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

$$
w = (X^T X)^{-1} X^T y
$$
    
    
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$ 

Выразим w' через:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y = (P^T (X^T X) P)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = P^{-1}(X^T X)^{-1} X^T y = P^{-1} w
$$ 

Рассчитаем a'.

$$
a' = XPw' = XPP^{-1} w = Xw = a
$$

Следовательно, при умножении на обратимую матрицу качество линейной регрессии не изменится.

### 3.2 Алгоритм
Для выполнения данного задания необходимо преобразовать данные клиента без потери качества машинного обучения.

1. Сгенерировать случайную квадратную невырожденную матрицу-ключ.
1. Домножить матрицу признаков на обратимую матрицу той же размерности, которая будет генерироваться случайно.

**Обоснование**

Выше мы доказали что если умножить матрицу объект признак на случайно сгенерированную признаки, то для линейной регрессии качество прогноза не изменится.

### 3.3 Проверка
Выведем значение метрики R2 на исходной таблице.

In [8]:
results = pd.DataFrame(columns=[' '])

def result(x, name):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

    model = LinearRegression()
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    r2 = r2_score(y_test, predictions)
    results.loc[name] = [r2]
    print(name + ': ', r2)

In [9]:
result(x, 'Исходная таблица')

Исходная таблица:  0.38787396350591696


Сгенерируем случайную квадратную невырожденную матрицу-ключ.

In [10]:
n = x.shape[1]
key = np.random.rand(n, n)

while np.linalg.det(key) == 0:
    key = np.random.rand(n, n)

Домножим матрицу признаков на матрицу-ключ.

In [11]:
x_key = x @ key

Выведем значение метрики R2 на зашифрованной таблице.

In [12]:
result(x_key, 'Зашифрованная таблица')

Зашифрованная таблица:  0.38787396350472925


Посмотрим на все результаты в одной таблице.

In [13]:
results

,
Исходная таблица,0.387874
Зашифрованная таблица,0.387874


<div style="border:solid steelblue 5px; padding: 30px">
   <p>
        <b>Качество линейной регрессии не отличается до и после преобразования.</b>
   </p>  
</div>

## 4. Общий вывод

<div style="border:solid steelblue 5px; padding: 30px">
   <p>
        <b>Проделано:</b>
        <ol><li>Данные подготовлены.</li>
        <li>Предложен алгоритм преобразования данных.</li>
        <li>Алгоритм проверен.</li>
        </ol>
        <b>Выявлено:</b>
        <li>При умножении на обратимую матрицу качество линейной регрессии не изменится.</li>
        <li>Качество линейной регрессии не отличается до и после преобразования.</li>
   </p>  
</div>